In [ ]:
!pip3 install geopy

In [ ]:
!ls -alt
!pwd
!date

In [ ]:
from geopy.geocoders import Nominatim
from geopy.adapters import AioHTTPAdapter
import pandas as pd
import time

df = pd.read_csv("./Map_Chart_Datasets/filtered_BexarMay25_hotel_tax_receipts.csv")

# Point to your local server
geolocator = Nominatim(
    user_agent="tx_hotels_map",
    domain="localhost:8080",
    scheme="http"
)

def get_coordinates(row):
    full_address = f"{row['Location_Address']}, {row['Location_City']}, TX {row['Location_Zip']}"
    try:
        location = geolocator.geocode(full_address, timeout=3)
#        time.sleep(1)  # Respect rate limits even locally
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"Error with address '{full_address}': {e}")
        return pd.Series([None, None])

df[["Latitude", "Longitude"]] = df.apply(get_coordinates, axis=1)


In [ ]:
df.info()

In [ ]:
!pip3 install folium

In [ ]:
!date

In [ ]:
import folium

# Create the base map centered on San Antonio
hotel_map = folium.Map(location=[29.5187, -98.6047], zoom_start=12)

# Normalize receipts to set radius scale
min_receipt = df["Total_Room_Receipts"].min()
max_receipt = df["Total_Room_Receipts"].max()
range_receipt = max_receipt - min_receipt

# Add scaled circle markers
for _, row in df.dropna(subset=["Latitude", "Longitude"]).iterrows():
    # Scale radius between 4 and 20
    scaled_radius = 4 + 16 * ((row["Total_Room_Receipts"] - min_receipt) / range_receipt)

    # Color by receipt amount (simple logic: low = light, high = dark blue)
    if row["Total_Room_Receipts"] == 0:
        color = "gray"
    elif row["Total_Room_Receipts"] < 100000:
        color = "lightblue"
    elif row["Total_Room_Receipts"] < 300000:
        color = "blue"
    else:
        color = "darkblue"

    popup_text = f"""
    <b>{row['Location_Name']}</b><br>
    Receipts: ${row['Total_Room_Receipts']:,.2f}<br>
    Units: {row['Unit_Capacity']}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=scaled_radius,
        color=color,
        fill=True,
        fill_color=color,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(hotel_map)

# Save the map
hotel_map.save("Bexar_May25_receipts_scaled_map_2.html")


In [ ]:
!date

In [ ]:
!ls
!pwd